- https://spacy.io/
- Resume parsing technology converts an unstructured form of resume data into a structured format.
- A Resume Parser helps organizations eliminate the error-prone and time-consuming process and improves recruiters' efficiency.
- We will use Spacy and Transformers for training and prediction

## NER Training Preparation
- Download base `config` file from https://spacy.io/usage/training#quickstart
- Modify config file for training data
- Prepare training data
- Do training
- Final testing

Data is gotten from:
- https://github.com/laxmimerit
- https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

## Data Annotation Tools

- doccano https://doccano.herokuapp.com/
- labelstudio https://labelstud.io/
- INCEpTION https://webanno.github.io/webanno/

In [ ]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 38.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
#Import packages
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__ #check spacy version

'3.7.4'

In [ ]:
!nvidia-smi

Thu Apr 25 14:52:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git #clone repo for our data

fatal: destination path 'CV-Parsing-using-Spacy-3' already exists and is not an empty directory.


In [ ]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', 'r')) #load train data

In [ ]:
len(cv_data)

200

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# cv_data[0]

In [ ]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train), len(test)

(140, 60)

In [ ]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 60/60 [00:00<00:00, 86.90it/s]


In [ ]:
len(db.tokens)

60

In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 165kB/s]
config.json: 100% 481/481 [00:00<00:00, 3.61MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 9.81MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 45.6MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 34.5MB/s]
model.safetensors: 100% 499M/499M [00:03<00:00, 128MB/s]
Some weights of R

### Model Test

In [ ]:
nlp = spacy.load('/content/output/model-best')


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
doc = nlp('my name is Emmanuel Ezeokeke. I worked at Amazon. I have 10 years of experience')
for ent in doc.ents:
  print(ent.text, "   ->>>>> ", ent.label_)

Emmanuel Ezeokeke    ->>>>>  Name
Amazon    ->>>>>  Companies worked at


In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 37.4 MB/s eta 0:00:00


In [ ]:
import sys, fitz

In [ ]:
fname = '/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)

In [ ]:
# doc = [page.getText() for page in doc]

In [ ]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [ ]:
text = text.strip()

In [ ]:
text = ' '.join(text.split())

In [ ]:
text

'Alice Clark AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development • Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL) Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searching with Bing, browsing with Microsoft Edg

In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, "   ->>>>> ", ent.label_)

Alice Clark    ->>>>>  Name
Delhi    ->>>>>  Location
Microsoft    ->>>>>  Companies worked at
Software Engineer    ->>>>>  Designation
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Indian Institute of Technology    ->>>>>  College Name
Machine Learning, Natural Language Processing, and Big Data Handling ADDITIONAL INFORMATION Professional Skills • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal skills with ability to interact with individuals at all the levels • Quick learner and maintains cordial relationship with project manager and team members and good performer both in team and independent job environments • Positive attitude towards superiors &amp; peers • Supervised junior developers throughout proj